In [1]:
import asyncio
from pathlib import Path
import logging
import sys
from typing import List, Optional

# Add project root to Python path if needed
project_root = str(Path().resolve().parent)
if project_root not in sys.path:
    sys.path.append(project_root)

from tqdm import tqdm
import pandas as pd
from FileUtils import FileUtils, OutputFileType

from src.semantic_analyzer import SemanticAnalyzer

# Configure logging
# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
# )
# logger = logging.getLogger(__name__)
from src.nb_helpers.environment import setup_notebook_env, verify_environment




In [ ]:
# Set up environment and verify
setup_notebook_env(log_level="WARNING")
verify_environment()



2024-12-12 18:54:36,486 - FileUtils.core.file_utils - INFO - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer


2024-12-12 18:54:36,486 - FileUtils.core.file_utils - INFO - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer


2024-12-12 18:54:36,497 - FileUtils.core.file_utils - INFO - FileUtils initialized with local storage


2024-12-12 18:54:36,497 - FileUtils.core.file_utils - INFO - FileUtils initialized with local storage


2024-12-12 18:54:36,551 - FileUtils.core.file_utils - INFO - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer


2024-12-12 18:54:36,551 - FileUtils.core.file_utils - INFO - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer


2024-12-12 18:54:36,558 - FileUtils.core.file_utils - INFO - FileUtils initialized with local storage


2024-12-12 18:54:36,558 - FileUtils.core.file_utils - INFO - FileUtils initialized with local storage


Environment Check Results:

Basic Setup:
-----------
✓ Project root in path
✓ FileUtils initialized
✓ .env file loaded

Environment Variables:
---------------------
✓ OPENAI_API_KEY set
✓ ANTHROPIC_API_KEY set

Project Structure:
-----------------
✓ Raw data exists
✓ Processed data exists
✓ Configuration exists
✓ Main config.yaml exists

Environment Status: Ready ✓


True

In [3]:
def display_analysis_summary(results: pd.DataFrame) -> None:
    """Display formatted analysis summary."""
    print("\nAnalysis Results Summary")
    print("=" * 50)

    # Basic stats
    print(f"Total rows processed: {len(results)}")
    if "language" in results.columns:
        print(f"Language: {results['language'].iloc[0]}")
    if "processing_time" in results.columns:
        print(
            f"Average processing time: {results['processing_time'].mean():.2f}s"
        )

    # Results by type
    result_sections = {
        "Keywords": [
            col for col in results.columns if col.startswith("keywords_")
        ],
        "Themes": [col for col in results.columns if col.startswith("themes_")],
        "Categories": [
            col for col in results.columns if col.startswith("categories_")
        ],
    }

    for section_name, columns in result_sections.items():
        if columns:
            print(f"\n{section_name} Results")
            print("-" * 50)

            # Display first 3 rows for each result type
            sample_results = results[columns].head(3)

            # Format each cell for display
            for idx, row in sample_results.iterrows():
                print(f"\nRow {idx + 1}:")
                for col in columns:
                    value = row[col]
                    if pd.notna(value):
                        print(f"  {col.split('_', 1)[1]}: {value}")


async def analyze_excel_content(
    content_file: str = "test_content_en.xlsx",
    parameter_file: str = "parameters_en.xlsx",
    content_column: str = "content",
    analysis_types: Optional[List[str]] = None,
    batch_size: int = 10,
) -> None:
    """Run Excel-based analysis with progress reporting."""
    try:
        # Initialize FileUtils
        file_utils = FileUtils()

        # Verify paths with progress
        with tqdm(total=2, desc="Checking files") as pbar:
            content_path = file_utils.get_data_path("raw") / content_file
            pbar.update(1)

            param_path = file_utils.get_data_path("parameters") / parameter_file
            pbar.update(1)

            if not content_path.exists():
                raise FileNotFoundError(
                    f"Content file not found: {content_path}"
                )
            if not param_path.exists():
                raise FileNotFoundError(
                    f"Parameter file not found: {param_path}"
                )

        print("\nStarting analysis:")
        print(f"Content file: {content_path}")
        print(f"Parameter file: {param_path}")
        print(f"Analysis types: {analysis_types or 'all'}")

        # Create and run analyzer
        analyzer = SemanticAnalyzer.from_excel(
            content_file=content_file,
            parameter_file=parameter_file,
            content_column=content_column,
            file_utils=file_utils,
        )

        # Run analysis with progress
        results = await analyzer.analyze_excel(
            analysis_types=analysis_types,
            batch_size=batch_size,
            save_results=True,
            output_file="analysis_results",
            show_progress=True,
        )

        # Display formatted results
        display_analysis_summary(results)

    except Exception as e:
        logger.error(f"Analysis failed: {e}", exc_info=True)




In [4]:
if __name__ == "__main__":
    print("Running Excel-based analysis...")

    # await run(
    await analyze_excel_content(
        content_file="test_content_fi.xlsx",
        parameter_file="parameters_fi.xlsx",
        analysis_types=["keywords", "themes"],
    )
    # )


Running Excel-based analysis...
2024-12-12 18:54:36,696 - FileUtils.core.file_utils - INFO - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer


2024-12-12 18:54:36,696 - FileUtils.core.file_utils - INFO - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer


2024-12-12 18:54:36,703 - FileUtils.core.file_utils - INFO - FileUtils initialized with local storage


2024-12-12 18:54:36,703 - FileUtils.core.file_utils - INFO - FileUtils initialized with local storage
Checking files: 100%|██████████| 2/2 [00:00<00:00, 2000.14it/s]
2024-12-12 18:54:36,712 - src.semantic_analyzer.analyzer - INFO - Initializing Excel Semantic Analyzer...



Starting analysis:
Content file: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\raw\test_content_en.xlsx
Parameter file: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\parameters\parameters_en.xlsx
Analysis types: ['keywords', 'themes']


...


Raw LLM response: {'themes': [{'name': 'Cloud Computing Services', 'description': 'The provision of scalable infrastructure that supports various deployments in a cloud environment.', 'confidence': 0.95, 'keywords': ['cloud', 'computing', 'scalable', 'infrastructure', 'deployment'], 'domain': 'technical/business'}, {'name': 'Microservices Architecture', 'description': 'A design approach that promotes modularity and maintainability in system development, allowing for independent deployment of services.', 'confidence': 0.9, 'keywords': ['microservices', 'architecture', 'modular', 'maintainable', 'system', 'design'], 'domain': 'technical/business'}, {'name': 'API Management', 'description': 'The handling of authentication and data validation through API endpoints, ensuring secure and reliable communication between services.', 'confidence': 0.85, 'keywords': ['API', 'endpoints', 'authentication', 'data', 'validation', 'requirements'], 'domain': 'technical/business'}], 'evidence': {'Cloud 

2024-12-12 18:55:08,472 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-12 18:55:08,476 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-12 18:55:08,491 - src.analyzers.excel_support - INFO - Theme analysis complete
Analysis Progress: 100%|██████████| 2/2 [00:20<00:00, 10.13s/it]
2024-12-12 18:55:08,499 - src.semantic_analyzer.analyzer - INFO - Combining results...
2024-12-12 18:55:08,512 - src.semantic_analyzer.analyzer - INFO - Saving results...
2024-12-12 18:55:08,586 - LocalStorage - INFO - Saved Excel file with sheets: ['Analysis Results']
2024-12-12 18:55:08,586 - LocalStorage - INFO - Saved Excel file with sheets: ['Analysis Results']



Raw LLM response: {'themes': [{'name': 'Integration of Theory and Practice', 'description': 'The theme emphasizes the combination of theoretical concepts with practical exercises in classroom workshops, highlighting the importance of applying knowledge in real-world scenarios.', 'confidence': 0.95, 'keywords': ['theoretical', 'exercise', 'combine', 'hands-on', 'workshop'], 'domain': 'technical/business', 'parent_theme': None}, {'name': 'Peer Learning and Collaboration', 'description': 'This theme focuses on the role of small group activities in promoting peer learning and knowledge sharing among participants, fostering a collaborative learning environment.', 'confidence': 0.9, 'keywords': ['peer', 'group', 'activity', 'share', 'knowledge'], 'domain': 'technical/business', 'parent_theme': None}, {'name': 'Reinforcement of Learning Objectives', 'description': 'The theme highlights the importance of practice sessions in reinforcing key learning objectives, ensuring that participants effe

2024-12-12 18:55:08,589 - FileUtils.core.file_utils - INFO - Data saved successfully: {'analysis_results_20241212_185508': 'c:\\Users\\tja\\OneDrive - Rastor-instituutti ry\\Tiedostot\\Rastor-instituutti\\kehittäminen\\analytiikka\\repos\\semantic-text-analyzer\\data\\processed\\analysis_results_20241212_185508.xlsx'}
2024-12-12 18:55:08,592 - src.semantic_analyzer.analyzer - INFO - Saved results to: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\processed\analysis_results_20241212_185508.xlsx
2024-12-12 18:55:08,595 - src.semantic_analyzer.analyzer - INFO - Results saved to: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\processed\analysis_results_20241212_185508.xlsx
2024-12-12 18:55:08,596 - src.semantic_analyzer.analyzer - INFO - Analysis completed in 20.37 seconds



Analysis Results Summary
Total rows processed: 9
Language: en
Average processing time: 20.29s

Keywords Results
--------------------------------------------------

Row 1:
  id: technical_1
  type: technical
  language: en
  keyword_scores: 0.95, 0.95, 0.90, 0.90, 0.90, 0.85, 0.85, 0.80
  keyword_domains: technical, technical, technical, technical, technical, technical, technical, technical

Row 2:
  id: technical_2
  type: technical
  language: en
  keyword_scores: 0.95, 0.95, 0.95, 0.90
  keyword_domains: technical, technical, technical, technical

Row 3:
  id: technical_3
  type: technical
  language: en
  keyword_scores: 0.95, 0.95, 0.90, 0.90, 0.90, 0.90
  keyword_domains: technical, technical, technical, technical, technical, technical

Themes Results
--------------------------------------------------

Row 1:
  id: technical_1
  type: technical
  language: en
  theme_descriptions: The systematic approach to training machine learning models using large datasets, emphasizing the im